In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/RA/robust_yolov5/yolov5

%load_ext autoreload
%autoreload 2

Mounted at /content/drive
/content/drive/MyDrive/RA/robust_yolov5/yolov5


In [2]:
!pip install -qr requirements.txt 

     |████████████████████████████████| 1.6 MB 5.2 MB/s 
     |████████████████████████████████| 1.6 MB 47.4 MB/s 
     |████████████████████████████████| 615 kB 74.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


$RECYCLE.BIN                          __init__.py
Copy of Copy of pcap_processing.ipynb __pycache__
Data_from_external_database           data_to_pcd
Ignore                                pcap_processing.ipynb
New_Data                              pcap_to_pcd.ipynb
RA                                    pcap_transformation.ipynb
SBIR                                  pcap_util.py
Sample_Data                           pcd
SplitCap.exe                          requirements.txt
System Volume Information             video.ipynb


In [58]:
from ouster import client
from ouster import pcap
from contextlib import closing
from os import listdir
import subprocess
import os

from tqdm import tqdm
import cv2
import numpy as np
from joblib import Parallel, delayed

In [4]:
%pylab inline
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [ ]:
def transform_pcap(meta_path,file_path,save_path):

  with open(meta_path, 'r') as f:
    metadata = client.SensorInfo(f.read())

  pcap_file = pcap.Pcap(file_path, metadata)

  fps = int(str(metadata.mode)[-2:])
  width = int(str(metadata.mode)[:4])
  height = int(str(metadata.prod_line)[5:])

  left_clip = 190
  right_clip = 275

  width -= left_clip + right_clip

  with closing(client.Scans(pcap_file)) as scans:

    vid_writer_sig = cv2.VideoWriter(save_path+"_signal.avi", cv2.VideoWriter_fourcc(*"RGBA"), fps, (width, height))
    vid_writer_range = cv2.VideoWriter(save_path+"_range.avi", cv2.VideoWriter_fourcc(*"RGBA") , fps, (width, height))
    vid_writer_ref = cv2.VideoWriter(save_path+"_reflectivity.avi", cv2.VideoWriter_fourcc(*"RGBA"), fps, (width, height))
    vid_writer_nir = cv2.VideoWriter(save_path+"_near_ir.avi", cv2.VideoWriter_fourcc(*"RGBA") , fps, (width, height))


    for scan in tqdm(scans, position=0, leave=True):

      ref_field = scan.field(client.ChanField.REFLECTIVITY)
      ref_val = client.destagger(pcap_file.metadata, ref_field)
      ref_img = ref_val.astype(np.uint8)[:,left_clip:-right_clip]

      sig_field = scan.field(client.ChanField.SIGNAL)
      sig_val = client.destagger(pcap_file.metadata, sig_field)
      sig_img = sig_val.astype(np.uint8)[:,left_clip:-right_clip]

      ir_field = scan.field(client.ChanField.NEAR_IR)
      ir_val = client.destagger(pcap_file.metadata, ir_field)
      ir_img = ir_val.astype(np.uint8)[:,left_clip:-right_clip]

      range_field = scan.field(client.ChanField.RANGE)
      range_val = client.destagger(pcap_file.metadata, range_field)
      range_img = range_val.astype(np.uint8)[:,left_clip:-right_clip]
      
      sig_img = np.dstack((sig_img, sig_img, sig_img))
      range_img = np.dstack((range_img, range_img, range_img))
      ref_img = np.dstack((ref_img, ref_img, ref_img))
      ir_img = np.dstack((ir_img, ir_img, ir_img))
      
      vid_writer_sig.write(sig_img)    
      vid_writer_range.write(range_img)
      vid_writer_ref.write(ref_img)    
      vid_writer_nir.write(ir_img)

    vid_writer_sig.release()
    vid_writer_range.release()
    vid_writer_ref.release()
    vid_writer_nir.release()
    cv2.destroyAllWindows()


In [86]:
prefix = "../../data/"
dates = {"SBIR_2022-10-08\\":(24,30),"SBIR_2022-10-07\\":(15,20),"SBIR_2022-09-24\\":(1,18),"SBIR_2022-10-06\\":(1,14),"SBIR_2022-10-21\\":(37,40),"SBIR_2022-10-22\\":(41,53),"SBIR_2022-10-20\\":(33,36)}

In [ ]:
for date in dates:
  meta_path = prefix + date + "meta.json"
  print(f"In {date}")
  for i in range(1,dates[date]+1):

    pcap_path = prefix + date + str(i) + ".pcap"

    save_path = prefix + date + str(i)

    print(f"Processing {pcap_path}")

    transform_pcap(meta_path,pcap_path,save_path)

In [ ]:
def process_given_date(date,val):
  prefix = "../../new_data/" + date + f"{val}/"

  files = [f for f in listdir(prefix)]

  meta_path = prefix + files[0][:-5] + ".json"
  
  pcap_path = prefix + files[0][:-5] + ".pcap"

  save_path = prefix + files[0][:-5]

  print(f"Started {prefix}...")

  # transform_pcap(meta_path,pcap_path,save_path)

  subprocess.run(["tcpdump", "-r", pcap_path, "-w", "small_file.pcap", "-C", "2000"])

  print(f"Finished {prefix}...")
  
  # for i in range(1,val+1):

  #   pcap_path = prefix + date + str(i) + ".pcap"

  #   save_path = prefix + date + str(i)

  #   print(f"Processing {pcap_path}")

  #   transform_pcap(meta_path,pcap_path,save_path)

In [ ]:
date = "SBIR_2022-09-24/"
start,end = dates[date]

In [ ]:
process_given_date(date,1)

In [ ]:
Parallel(n_jobs=4, prefer="threads")(delayed(process_given_date)(date,val) for val in range(start,end+1))

In [ ]:
transform_pcap("../../data/14_31/meta.json","../../data/14_31/1.pcap","./transformed/1")

1255it [00:45, 27.62it/s]


In [6]:
!apt-get update
!apt install tcpdump

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.c

In [7]:
def break_up_pcap(date,val):
  prefix = "../../new_data/" + date + f"{val}/"

  files = [f for f in listdir(prefix)]

  meta_path = prefix + files[0][:-5] + ".json"
  
  pcap_path = prefix + files[0][:-5] + ".pcap"

  save_path = prefix + files[0][:-5]

  print(f"Started {pcap_path}...")

  # transform_pcap(meta_path,pcap_path,save_path)

  subprocess.run(["tcpdump", "-r", pcap_path, "-w", f"{prefix}small_file.pcap", "-C", "2000"])

  print(f"Finished {prefix}...")
  
  # for i in range(1,val+1):

  #   pcap_path = prefix + date + str(i) + ".pcap"

  #   save_path = prefix + date + str(i)

  #   print(f"Processing {pcap_path}")

  #   transform_pcap(meta_path,pcap_path,save_path)

In [89]:
date = "SBIR_2022-10-20\\"

In [ ]:
break_up_pcap(date,1)

Started ../../new_data/SBIR_2022-09-24/1/2022-09-21-09-27-45_OS-0-128-992133001099-1024x10.pcap...
Finished ../../new_data/SBIR_2022-09-24/1/...


In [9]:
Parallel(n_jobs=2, prefer="threads")(delayed(break_up_pcap)(date,val)for val in [17,18]) #+list(range(14,19))

Started ../../new_data/SBIR_2022-09-24/18/2022-09-24-17-09-20_OS-0-128-992133001099-1024x10.pcap...
Started ../../new_data/SBIR_2022-09-24/17/2022-09-24-16-28-34_OS-0-128-992133001099-1024x10.pcap...
Finished ../../new_data/SBIR_2022-09-24/18/...
Finished ../../new_data/SBIR_2022-09-24/17/...


[None, None]

In [63]:
def rename_pcap(date,val):
  prefix = "D:\\SBIR\\" + date + f"{val}\\small_files\\"

  files = [f for f in listdir(prefix)]
  
  files = sorted(files,key = lambda e : int(e[e.find("Seconds_")+8:-5]) )
  
  print(f"Started {date}\{val}")
  for i,e in enumerate(files):
    renamed = f"{i+1}.pcap"
    # print("ren", prefix + e, prefix + renamed)
    os.rename(prefix + e, prefix + renamed)
  print(f"Finished {date}\{val}")

In [62]:
rename_pcap(date,1)

In [90]:
Parallel(n_jobs=2, prefer="threads")(delayed(rename_pcap)(date,val)for val in range(dates[date][0],dates[date][1]+1)) #range(dates[date][0],dates[date][1]+1)

Started SBIR_2022-10-20\\34
Started SBIR_2022-10-20\\33
Finished SBIR_2022-10-20\\34
Started SBIR_2022-10-20\\35
Finished SBIR_2022-10-20\\33
Started SBIR_2022-10-20\\36
Finished SBIR_2022-10-20\\36
Finished SBIR_2022-10-20\\35


[None, None, None, None]

In [36]:
%ls

 Volume in drive D is Raw
 Volume Serial Number is FCF1-1398

 Directory of D:\

11/12/2022  07:01 PM    <DIR>          .ipynb_checkpoints
11/10/2022  01:43 PM    <DIR>          .Trash-1000
11/12/2022  09:14 PM            30,553 Copy of Copy of pcap_processing.ipynb
11/12/2022  02:47 PM            16,195 Copy of pcap_processing.ipynb
11/12/2022  09:14 PM            14,319 pcap_processing.ipynb
11/12/2022  07:16 PM             1,336 requirements.txt
11/10/2022  01:55 PM    <DIR>          SBIR
11/12/2022  08:04 PM           512,696 SplitCap.exe
               5 File(s)        575,099 bytes
               3 Dir(s)  2,566,437,994,496 bytes free
